In [ ]:
from snowflake.snowpark.context import get_active_session
from datetime import date
session = get_active_session()

In [ ]:
py_db = session.get_current_database()
py_int_sch = session.get_current_schema()
py_env = 'DEV'if 'DEV' in session.get_current_schema() else 'PROD'
py_raw_sch = 'DEV_FRED_RAW' if 'DEV' in session.get_current_schema() else 'PROD_FRED_RAW'
py_har_sch = 'DEV_FRED_HARMONIZED' if 'DEV' in session.get_current_schema() else 'PROD_FRED_HARMONIZED'
py_ana_sch = 'DEV_FRED_ANALYTICS' if 'DEV' in session.get_current_schema() else 'PROD_FRED_ANALYTICS'
py_tgt = '{}.{}.FREDDATA'.format(py_db, py_har_sch)
py_src = '{}.{}.FREDDATA_STREAM'.format(py_db, py_raw_sch)
py_task = '{}.{}.DATA_UPDATE_TASK'.format(py_db, py_int_sch)
py_notebook = '{}.{}.{}_LOAD_FROM_STAGE()'.format(py_db, py_raw_sch, py_env)
py_view_src = '{}.{}.freddata'.format(py_db, py_ana_sch)
py_month_view = '{}.{}.month_wise_analytics'.format(py_db, py_ana_sch)
py_year_view = '{}.{}.year_wise_analytics'.format(py_db, py_ana_sch)
py_val_diff_view = '{}.{}.curr_vs_prev_analytics'.format(py_db, py_ana_sch)

In [ ]:
CREATE OR REPLACE FUNCTION calculate_percent_profit_loss(curr FLOAT, prev FLOAT) 
RETURNS FLOAT 
LANGUAGE SQL 
AS 
$$
    CASE 
        WHEN prev = 0 THEN 100
        ELSE trunc(((curr - prev) / prev) * 100,2)
    END
$$;

In [ ]:
CREATE OR REPLACE PROCEDURE FRED_DATA_UPDATE_SP(py_tgt STRING, py_src STRING)
RETURNS INT
LANGUAGE SQL 
EXECUTE AS CALLER
AS 
$$
BEGIN
    EXECUTE IMMEDIATE 
    'MERGE INTO ' || py_tgt || ' AS tgt
            USING (SELECT DISTINCT * FROM ' || py_src || ') AS src 
            ON tgt.DATA_DATE = src.DATA_DATE
            WHEN MATCHED AND src.METADATA$ACTION = ''INSERT'' THEN 
                UPDATE SET tgt.VALUE = src.VALUE, 
                        tgt.CREATED_DATE = CURRENT_TIMESTAMP 
            WHEN NOT MATCHED AND src.METADATA$ACTION = ''INSERT'' THEN 
                INSERT (DATA_DATE, VALUE, CREATED_DATE) 
                VALUES (src.DATA_DATE, src.VALUE, CURRENT_TIMESTAMP);';

    RETURN 1;
END;
$$;


In [ ]:
CREATE OR REPLACE TASK {{py_task}}
WAREHOUSE = FRED_WH
SCHEDULE = 'USING CRON 30 23 * * * UTC'
AS 
EXECUTE NOTEBOOK {{py_notebook}}

In [ ]:
ALTER TASK {{py_task}} RESUME;

In [ ]:
create or replace view {{py_val_diff_view}} as
select f.data_date as date, f.value as current_value, 
nvl(lag(value) over( order by data_date), 0) previous_day_value, 
concat(fred_db.prod_integrations.CALCULATE_PERCENT_PROFIT_LOSS(current_value, previous_day_value),' %') as PERCENT_DIFFERNCE from {{py_view_src}} as f;


In [ ]:
create or replace view {{py_year_view}} as
select year(f.data_date) as year, trunc(avg(f.value),2) as average_value from {{py_view_src}} f
group by year(f.data_date);

In [ ]:
CREATE OR REPLACE VIEW {{py_month_view}} AS
SELECT 
    year, 
    month,
    average_value
FROM (
    SELECT 
        YEAR(f.data_date) AS year,
        MONTH(f.data_date) AS month_num,
        MONTHNAME(f.data_date) AS month,
        TRUNC(AVG(f.value), 2) AS average_value
    FROM 
        {{py_view_src}} f
    GROUP BY 
        YEAR(f.data_date),
        MONTH(f.data_date),
        MONTHNAME(f.data_date)
) AS fred;